
<img align="left" src = https://www.linea.org.br/wp-content/themes/LIneA/imagens/logo-header.png width=120 style="padding: 20px"> <br> 
<img align="left" src = https://jupyter.org/assets/homepage/hublogo.svg width=200 style="padding: 20px"> <br> 
<br>
<br>
<br>
<br>

# JupyterHub Tutorial 
## Exemplos de aplicações - Métodos de Estatística

**Autor:** Michel Aguena, LAPP/IN2P3 & LIneA


Última verificação: 06/02/2023


***

Este notebook foi criado originalmente para fazer parte das aulas do Curso básico de Python aplicado à Astronomia, oferecido pelo LIneA em 2020.

## Objetivo
 
Introduzir conceitos básicos de estatística, estatística Bayseana e métodos para vinculação de parâmetros.

## Índice
1. [Calibrando uma relação (fazendo um "fit")](#fit)
2. [$\chi^2$ e *Likelihood* ($\mathcal{L}$)](#chi2like)
3. [O espaço de parâmetros](#param)

***


## Imports e setup

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
# computation
import numpy as np
from astropy.table import Table
# display
from IPython.display import Markdown as md
from IPython.display import display, Math
# plots
import pylab as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import MultipleLocator
# widgets plot
import ipywidgets as widgets
%matplotlib widget
# auto reload local modules
%load_ext autoreload
%autoreload 2
# my functions
import aux_funcs

# 1. Calibrando uma relação (fazendo um "fit") <a class="anchor" id="fit"></a>

Problema: Como definir quais valores de uma função se ajustam melhor um conjunto de dados?

Ex:

In [ ]:
x = np.array([1, 2, 3, 4, 5])
y = np.array([3, 5, 7, 9, 11])
err = np.array([3.12, 7.40, 5.61, 1.48 , 3.50])

In [ ]:
%matplotlib inline
plt.errorbar(x, y, err, ls='', lw=1, fmt='o', capsize=5, color="k")
plt.grid()
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Qual a melhor função que descreve esses dados?
Uma reta:
\begin{equation}
f(x) = a\; x + b
\end{equation}

In [ ]:
def func(x, a, b):
    return a*x+b

Que tal testar alguns parâmetros:

In [ ]:
%matplotlib widget
aux_funcs.plot_with_line(x, y, err, show_chi2=False)

In [ ]:
parametros = [(0, 7), (1, 2), (3, 0),
              (2.5, 1), (2, 6), (-3, 15)]
md('Dados alguns parâmetros:\n\n'+
   'Conjunto | a | b \n---|---|---\n'+
    '\n'.join([f'$c_{i}$ | {a} | {b}' for i, (a, b) in enumerate(parametros)])+
   '\n\nQual se ajusta melhor?'
  )

In [ ]:
%matplotlib inline
plt.errorbar(x, y, err, ls='', fmt='o', capsize=5, color="k")
for i, (a, b) in enumerate(parametros):
    plt.plot(x, func(x, a, b),
            zorder=0, label=f'$c_{i}$ (a={a}, b={b})')
plt.grid()
plt.legend(ncol=2)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Como avaliar quantativamente qual o melhor ajuste?

# 2. $\chi^2$ e *Likelihood* ($\mathcal{L}$)<a class="anchor" id="chi2like"></a>


Como avaliar quantativamente qual o melhor ajuste?

## Método do $\chi^2$:

\begin{equation}
\chi^2 = \sum_i \frac{(data_i-modelo_i)^2}{(erro_i)^2}
\end{equation}

In [ ]:
def chi2(data, modelo, erro):
    return sum((data-modelo)**2/erro**2)

In [ ]:
aux_funcs.show_chi2_numbers(x, y, err, parametros, func=func)

In [ ]:
%matplotlib inline
plt.errorbar(x, y, err, ls='', fmt='o', capsize=5, color="k")
for i, (a, b) in enumerate(parametros):
    plt.plot(x, func(x, a, b),
            zorder=0, label=f'$c_{i}$ ($\chi^2={chi2(y, func(x, a, b), err):.1f}$)')
plt.grid()
plt.legend(ncol=2)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
%matplotlib widget
aux_funcs.plot_with_line(x, y, err, func=func, show_chi2=True)

## Likelihood

A *likelihood* é como uma probablilidade no espaço de parâmetros. No caso de uma distribuição Gaussiana de dados, ela pode ser definida em termos do $\chi^2$:

\begin{equation}
\mathcal{L} = \frac{1}{\sqrt{2\pi \det({erro}^2)}} \exp{\left(-\frac{1}{2}\chi^2\right)}
=\frac{1}{\prod_i\sqrt{2\pi\,{erro}^2_i}}\exp{\left(-\frac{1}{2}\sum_i \frac{(data_i-modelo_i)^2}{(erro_i)^2}\right)}
\end{equation}

In [ ]:
def like_chi2(chi2_, err):
    return np.exp(-.5*chi2_)/np.prod(np.sqrt(2*np.pi)*err)
def like(data, modelo, erro):
    return like_chi2(chi2(data, modelo, erro), erro)

In [ ]:
md('Para o nosso conjunto de dados, temos:\n\n'+
   'Conjunto | $\chi^2$ | $\mathcal{L}$\n---|---|---\n'+
    '\n'.join([f'c{i}|${chi2(y, func(x, a, b), err):.0f}$|'+
                f'${np.exp(-chi2(y, func(x, a, b), err)/2):.2e}$'
               for i, (a, b) in enumerate(parametros)]))

# 3. O espaço de parâmetros <a class="anchor" id="param"></a>

Avaliar a qualidade do ajuste no espaço de parâmetros

In [ ]:
%matplotlib inline
for i, (a, b) in enumerate(parametros):
    plt.scatter(a, b, color=f'C{i}')
    plt.text(a, b, f'$c_{i}$({chi2(y, func(x, a, b), err):.0f})')
plt.xlabel('a')
plt.ylabel('b')
plt.grid()
plt.show()

E se calculassemos os valores na grade?

Definir  valores para a grade

In [ ]:
a_vals = np.linspace(-3, 7, 101)
b_vals = np.linspace(-14, 16, 99)
a_grid, b_grid, chi2_grid = aux_funcs.compute_chi2_grid(a_vals, b_vals, func, x, y, err)

* Grafico 3D do $\chi^2$:

In [ ]:
%matplotlib widget
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(a_grid, b_grid, chi2_grid,
                 lw=.5)

for i, (a, b) in enumerate(parametros): 
    ax.scatter(a, b, chi2(y, func(x, a, b), err),
               color=f'C{i}', label=f'$c_{i}$')

ax.set_xlabel('a')
ax.set_ylabel('b')
ax.set_zlabel('$\chi^2$')

ax.legend(ncol=2)

fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.resizable = True

plt.show()

* Grafico 3D da $\mathcal{L}$:

In [ ]:
%matplotlib widget

like_grid = like_chi2(chi2_grid, err)

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(a_grid, b_grid, like_grid,
                 lw=.5)

ax.set_xlabel('a')
ax.set_ylabel('b')
ax.set_zlabel('$\mathcal{L}$')

for i, (a, b) in enumerate(parametros): 
    ax.scatter(a, b, like(y, func(x, a, b), err),
               color=f'C{i}', label=f'c{i}')


fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.resizable = True

plt.show()

## 3.1 Fixar *VS* marginalizar parâmetros
Como encontrar definitivamente qual conjunto de parâmetros que melhor se ajusta aos dados?

### 3.1.1 Fixar parâmetros

Deteminar a likelihood de um parâmetro fixando-se o valor do outro parâmetro:

\begin{equation}
\mathcal{L}_f(a) = \mathcal{L}(a, b=b_0)
\end{equation}

O que acontece quando se fixa um dos parâmetros:

In [ ]:
%matplotlib inline
f, axes = plt.subplots(1, 2, figsize=(8, 4))

vals = np.linspace(0, 5, 100)
for b in [-4, -2, 0, 2, 4]:
    axes[0].plot(vals, [like(y, func(x, a, b), err) for a in vals],
            label=f'b={b}')

vals = np.linspace(-7, 10, 100)
for a in [1, 1.5, 2, 2.5, 3]:
    axes[1].plot(vals, [like(y, func(x, a, b), err) for b in vals],
            label=f'a={a}')
for ax in axes:
    ax.legend()
    ax.grid()
    ax.set_ylim(0, ax.get_ylim()[1])
axes[0].set_xlabel('a')
axes[1].set_xlabel('b')
axes[1].yaxis.tick_right()
plt.show()

In [ ]:
%matplotlib widget
aux_funcs.plot_like_marg(x, y, err, a_vals, b_vals)

### 3.1.2 Marginalizar parâmetros

Deteminar a likelihood de um parâmetro considerando-se todos os valores possíveis que o outro parâmetro pode assumir: 
\begin{equation}
\mathcal{L}_m(a) = \int_{-\infty}^{\infty} db\, \mathcal{L}(a, b)
\approx \sum_i \Delta b \mathcal{L}(a, b=b_i)
\end{equation}

Vamos comparar as likelihoods marginalizadas com as fixas na seção anterior:

In [ ]:
%matplotlib inline
f, axes = plt.subplots(2, figsize=(6, 9))

norm = lambda x, dx: np.array(x)/sum(np.array(x)*dx)

da, db = a_vals[1]-a_vals[0], b_vals[1]-b_vals[0]
axes[0].plot(a_vals, norm(np.sum(like_grid, axis=1), da), c='0', label='Marginalized')
axes[1].plot(b_vals, norm(np.sum(like_grid, axis=0), db), c='0', label='Marginalized')

axes[0].set_xlabel('a')
axes[0].xaxis.tick_top()
axes[0].xaxis.set_label_position('top') 

axes[1].set_xlabel('b')



for b in [-4, -2, 0, 2, 4]:
    vals = np.linspace(a_vals[0], a_vals[-1], 200)
    dv = (vals[1]-vals[0])
    axes[0].plot(vals, norm([like(y, func(x, a, b), err) for a in vals], dv),
                label=f'b={b}', zorder=0, lw=.8)
for a in [1, 1.5, 2, 2.5, 3]:
    vals = np.linspace(b_vals[0], b_vals[-1], 200)
    dv = (vals[1]-vals[0])
    axes[1].plot(vals, norm([like(y, func(x, a, b), err) for b in vals], dv),
                label=f'a={a}', zorder=0, lw=.8)
    
for ax in axes:
    ax.grid()
    ax.grid(which='minor', lw=.5)
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    ax.legend()
axes[0].set_xlim(-1, 5)
axes[1].set_xlim(-10, 10)
axes[1].xaxis.set_major_locator(MultipleLocator(5))
plt.show()

## 3.2 Degeneressencia

Há alguma correlação entre os parâmetros calibrados? O valor de um dos dois parâmetros influencia no valor do outro?


Vamos ver a dependencia da likelihood:

In [ ]:
%matplotlib inline
print(like_grid.min(), like_grid.max())
fig = plt.figure(figsize=(10, 7))
cb = plt.contourf(a_grid, b_grid, like_grid,
            levels=np.linspace(like_grid.min(), like_grid.max(), 100)
            )
plt.colorbar(cb)
plt.xlabel('a')
plt.ylabel('b')

Quais valores se ajustam melhor?

Qual a influência da barra de erro dos dados?

E se tivessemos menos dados?

In [ ]:
%matplotlib widget
aux_funcs.plot_fit_degeneressence(x, y, err, a_vals, b_vals)

In [ ]:
%matplotlib widget
aux_funcs.plot_fit_degenerated(x, y, err, a_vals, b_vals, 3)

A correlação entre os parâmetros se dá pelo envolvimento na likelihood.

Ex:

\begin{equation}
\mathcal{L} \propto \exp{\left[-(a^2+b^2+2\theta ab)\right]}
\end{equation}

In [ ]:
%matplotlib widget
aux_funcs.plot_like_deg()